In [17]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from typing import List

In [18]:
llm = OllamaFunctions(
    model='llama3',
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    format='json',
    temperature=1  # Set your desired temperature here
)


In [19]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

#In case of multiple diagnoses
class Diagnosis(BaseModel):
    name: str = Field(default=None, description="Diagnosis for the patient")

class Treatment(BaseModel):
    name: str = Field(default=None, description="Name of the patient's treatment")
    dosage: str = Field(default=None, description="Detailed information for the treatment's dosage")

class Vital(BaseModel):
    name: str = Field(default=None, description="Vital of the patient")
    value: str = Field(default=None, description="Value of the Vital")
    units: str = Field(default=None, description="Units of the Vital")

class Person(BaseModel):
    name: str = Field(default=None, description="Name of the patient", required=True)
    age: float = Field(default=None, description="Age of the patient")
    gender: str = Field(default=None, description="Gender of the patient")
    diagnosis: List[Diagnosis]
    treatment: List[Treatment] 
    vital: List[Vital]

prompt = PromptTemplate.from_template(
    """"    
System: You are an AI that summarizes medical conversations into a structured JSON format. Given the medical transcript below, provide a summary by extracting key-value pairs. Only use the information explicitly mentioned in the transcript, and do not infer or assume any details that are not directly stated.
Speaker 1: Good morning, Nurse. Could you please update me on the status of our patient, Mr. Anderson?
Speaker 2: Good morning, Doctor. Mr. Anderson's blood pressure has stabilized at 100/70. His blood glucose level this morning was 8 mmol/L.
Speaker 1: That's good to hear. Has he taken his medication for the morning?
Speaker 2: Yes, Doctor. He took his Metformin at 7 AM, as prescribed.
Speaker 1: Excellent. Please continue to monitor his vitals and let me know if there are any significant changes.
Speaker 2: Absolutely, Doctor. I'll keep you updated.
Human: {request}
AI: """
)

# Chain
structured_llm = llm.with_structured_output(Person)
chain = prompt | structured_llm

In [21]:
# Function to invoke the chain and handle errors
def invoke_chain_with_retry(content):
    success = False
    response = None
    while not success:
        try:
            # Attempt to invoke the chain
            response = chain.invoke(f"Strictly following this template and its value types: {content}")
            success = True  # If successful, set success to True to exit the loop
        except Exception as e:
            # If there is an error, print the error and retry
            print(f"Error encountered: {e}. Retrying...")
    
    return response


In [22]:
# Example content

content="""
{
    "tool": "Person",
    "tool_input": 
    {
        "name": "Mr. Smith",
        "age": 67,
        "gender": "Male",
        "diagnosis": [
            {
            "name": "Type 2 Diabetes"
            }
        ],
        "treatment": [
            {
            "name": "Metformin"
            "dosage": "None"
            }
        ],
        "vital": [
            {
            "name": "Blood Pressure",
            "value": "120/80"
            "units": "mmHg"
            },
            {
            "name": "Blood Glucose Level",
            "value": "7.8"
            "units": "mmol/L"
            }
        ]
    }
}
"""

# Invoke the chain with retry logic
response = invoke_chain_with_retry(content)
print(response)

{ "tool": "Person", "tool_input": 
    {
        "name": "Mr. Anderson",
        "age": null,
        "gender": null,
        "diagnosis": [
            {"name": null}
        ],
        "treatment": [
            {"name": "Metformin", "dosage": "7 AM"}
        ],
        "vital": [
            {
                "name": "Blood Pressure",
                "value": "100/70"
                ,"units": null
            },
            {
                "name": "Blood Glucose Level",
                "value": "8 mmol/L",
                "units": "mmol/L"
            }
        ]
    } 
}
 
 
 

 
 

 
 

 
 

 
 

 
 

name='Mr. Anderson' age=None gender=None diagnosis=[Diagnosis(name=None)] treatment=[Treatment(name='Metformin', dosage='7 AM')] vital=[Vital(name='Blood Pressure', value='100/70', units=None), Vital(name='Blood Glucose Level', value='8 mmol/L', units='mmol/L')]
